# 仿真 Pi 门的标定

## 1. 基本参数

In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import qutip
from qutip import basis, sigmax, sigmay, sigmaz, Result
import matplotlib.pyplot as plt

# 导入我们自己库中的所有组件
from quantum_sim import (
    DuffingOscillatorModel,
    MicrowaveSequence,
    Simulator,
    CosineEnvelope,
    plotting,
    DRAGCorrector,
    Scanner
)


## 2. 扫描驱动幅度

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result

# 假设所有必要的类都已从您的库中导入
# from quantum_sim import ...

# --- 1. 物理系统参数 ---
D_LEVELS = 15
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# --- 2. 脉冲参数 ---
PULSE_DURATION = 20 # ns, 单个 pi/2 脉冲的时长
PULSE_AMP_RAD_NS = np.pi / PULSE_DURATION # 理论 pi/2 脉冲峰值振幅

# --- 3. 初始状态 ---
PSI0 = basis(D_LEVELS, 0)

# --- 4. 创建模型和仿真器实例 (这些在所有扫描中共享) ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, 
                options={
                    'nsteps': 10000,
                    'atol': 1e-9,
                    'rtol': 1e-7,
                    'store_states': True
                }
            )

# --- 5. 定义多次扫描的参数 ---

# a) 我们要扫描的重复次数列表
repeat_counts = [2, 6, 10]
# repeat_counts = [2, 4]

# b) 幅度缩放因子的扫描范围 (对所有实验保持一致)
amp_scale_factors = np.linspace(0.75, 1.25, 31) 

# c) 定义计算观测量 (对所有实验保持一致)
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()

# d) 准备一个字典来存储每次扫描的结果
results_dict = {}

print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 6. 外层循环：遍历不同的脉冲重复次数 ---

for repeat_num in repeat_counts:
    print(f"\n===== 开始扫描 {repeat_num} 次脉冲重复的情况 =====")

    # a) 动态定义仿真时间轴，总时长随重复次数变化
    total_duration = PULSE_DURATION * repeat_num
    t_list = np.linspace(0, total_duration, int(1001 * repeat_num)) # 确保时间分辨率

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        't_list': t_list,
        'use_rwa': True,
        'e_ops': [pop1_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数 (关键：使用当前的 repeat_num)
    def amplitude_setup(scale_factor: float) -> dict:
        current_amp = PULSE_AMP_RAD_NS * scale_factor
        # 使用 repeat_num 参数来创建重复脉冲
        new_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=current_amp).repeat(repeat_num)
        new_mw_sequence = MicrowaveSequence(envelope=new_envelope, carrier_freq=model.omega_q, phi=np.pi/3)
        return {'sequences': {'XY_drive': new_mw_sequence}}

    # e) Post-process 函数保持不变
    def final_pop1_processor(result: Result) -> float:
        final_population_1 = result.expect[0][-1]
        return final_population_1

    # f) 执行单次完整的扫描
    scanned_scales, final_populations = scanner.run(
        sweep_values=amp_scale_factors,
        setup_function=amplitude_setup,
        post_process_function=final_pop1_processor,
    )

    # g) 将结果存储到字典中
    results_dict[repeat_num] = final_populations

print("\n...所有扫描完成。")

# --- 7. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, (repeat_num, populations) in enumerate(results_dict.items()):
    color = colors(i / len(results_dict))
    
    # 绘制扫描结果
    ax.plot(scanned_scales, populations, 'o-', color=color, label=f'Simulated P(|1>) for {repeat_num} pulse(s)')
    
    # 寻找并标记每次扫描的最优幅度
    optimal_scale_index = np.argmax(populations)
    optimal_scale = scanned_scales[optimal_scale_index]
    max_population = populations[optimal_scale_index]
    ax.plot(optimal_scale, max_population, '*', color=color, markersize=15, 
            label=f'Optimal for {repeat_num} pulse(s) (scale={optimal_scale:.3f})')

# 标记理论 pi 脉冲幅度的位置
ax.axvline(x=1.0, color='r', linestyle='--', label='Theoretical π-pulse amp (scale=1.0)')

# 设置图表格式
ax.set_xlabel("Amplitude Scaling Factor")
ax.set_ylabel("Final Population of |1> State")
ax.set_title(f"Pi Gate Calibration for Repeated Pulses (Base Duration = {PULSE_DURATION} ns)")
ax.grid(True)
ax.legend()
plt.show()




In [ ]:
# 综合多次扫描的结果，绘制平均值，并寻找最优幅度

fig, ax = plt.subplots(figsize=(10, 6))
average_population = np.average([value for value in results_dict.values()], axis=0)

best_scale_index = np.argmax(average_population)
best_scale = scanned_scales[best_scale_index]
best_population = average_population[best_scale_index]

plt.plot(scanned_scales, average_population, 'o-', color='blue', label='Average P(|1>) over all scans')
plt.plot(best_scale, best_population, '*', color='orange', markersize=15, 
         label=f'Optimal Average (scale={best_scale:.3f})')
plt.xlabel("Amplitude Scaling Factor")
plt.ylabel("Average Final Population of |1> State")
plt.title(f"Average Pi Gate Calibration (Base Duration = {PULSE_DURATION} ns)")
plt.grid(True)
plt.legend()
plt.show()


## 3. 扫描 DRAG alpha 系数

In [ ]:
# 基于刚刚扫到的最佳的参数 1.01，对 DRAG alpha 进行扫描
# 类似地，也要分多次，叠不同的门数进行扫描

import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result

# 假设所有必要的类都已从您的库中导入
# from quantum_sim import ...

# --- 1. 物理系统参数 ---
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# --- 2. 脉冲参数 ---
PULSE_DURATION = 20 # ns, 单个 pi 脉冲的时长
PULSE_AMP_RAD_NS = np.pi / PULSE_DURATION * best_scale # 基于最佳幅度的 pi 脉冲峰值振幅

# --- 3. 初始状态 ---
PSI0 = basis(D_LEVELS, 0)

# --- 4. 创建模型和仿真器实例 (这些在所有扫描中共享) ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, 
                options={
                    'nsteps': 10000,
                    'atol': 1e-9,
                    'rtol': 1e-7,
                    'store_states': True
                }
            )




# --- 5. 定义多次扫描的参数 ---

# a) 我们要扫描的重复次数列表
repeat_counts = [2, 22, 42]

# b) alpha 系数
scanned_alpha = np.linspace(-30, 30, 41) # 增加点数以获得更平滑的曲线

# c) 定义计算观测量 (对所有实验保持一致)
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
leakage_operator = sum(basis(D_LEVELS, i) * basis(D_LEVELS, i).dag() for i in range(2, D_LEVELS))

# d) 准备一个字典来存储每次扫描的结果
results_dict = {}

print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 6. 外层循环：遍历不同的脉冲重复次数 ---

for repeat_num in repeat_counts:
    print(f"\n===== 开始扫描 {repeat_num} 次脉冲重复的情况 =====")

    # cos_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS).repeat(repeat_num)
    # mw_seq_no_drag = MicrowaveSequence(envelope=cos_envelope, carrier_freq=model.omega_q)

    # a) 动态定义仿真时间轴，总时长随重复次数变化
    total_duration = PULSE_DURATION * repeat_num
    t_list = np.linspace(0, total_duration, int(1001 * repeat_num)) # 确保时间分辨率

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        't_list': t_list,
        'use_rwa': True,
        'e_ops': [pop1_operator, leakage_operator],
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数 (关键：使用当前的 repeat_num)
    def drag_setup(alpha: float) -> dict:
        # 使用 repeat_num 参数来创建重复脉冲
        cos_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS).repeat(repeat_num)
        mw_seq_no_drag = MicrowaveSequence(envelope=cos_envelope, carrier_freq=model.omega_q)

        drag_corrector = DRAGCorrector(anh=model.anh, alpha=alpha)
        mw_seq_with_drag = drag_corrector.apply(mw_seq_no_drag)
        return {'sequences': {'XY_drive': mw_seq_with_drag}}

    # e) Post-process 函数保持不变
    def final_pop1_processor(result: Result) -> float:
        final_population_1 = result.expect[0][-1]
        return final_population_1

    # f) 执行单次完整的扫描
    scanned_scales, final_populations = scanner.run(
        sweep_values=scanned_alpha,
        setup_function=drag_setup,
        post_process_function=final_pop1_processor,
    )

    # g) 将结果存储到字典中
    results_dict[repeat_num] = final_populations

print("\n...所有扫描完成。")

# --- 7. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, (repeat_num, populations) in enumerate(results_dict.items()):
    color = colors(i / len(results_dict))
    
    # 绘制扫描结果
    ax.plot(scanned_scales, populations, 'o-', color=color, label=f'Simulated P(|1>) for {repeat_num} pulse(s)')
    
    # 寻找并标记每次扫描的最优幅度
    optimal_alpha_index = np.argmax(populations)
    optimal_alpha = scanned_alpha[optimal_alpha_index]
    max_population = populations[optimal_alpha_index]
    ax.plot(optimal_alpha, max_population, '*', color=color, markersize=15, 
            label=f'Optimal for {repeat_num} pulse(s) (alpha={optimal_alpha:.3f})')


# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |1> State")
ax.set_title(f"Pi Gate Calibration for Repeated Pulses (Base Duration = {PULSE_DURATION} ns)")
ax.grid(True)
ax.legend()
plt.show()




## 4. APE 序列仿真

这部分内容主要是参考并复现以下文章：

> Lucero E, Kelly J, Bialczak R C, et al. Reduced phase error through optimized control of a superconducting qubit[J]. Physical Review A—Atomic, Molecular, and Optical Physics, 2010, 82(4): 042339.

查看波形序列的包络

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# 假设您的 pulses.py 文件在当前目录或 Python 路径下
from quantum_sim import CosineEnvelope, CompositeEnvelope

# --- 1. 定义单个 pi/2 脉冲的参数 ---
PULSE_DURATION = 20  # ns
PULSE_AMP = np.pi / PULSE_DURATION

# --- 2. 创建构成 APE 伪幺正操作的两个基础包络 ---
x_pos_env = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP)
x_neg_env = CosineEnvelope(duration=PULSE_DURATION, amp=-PULSE_AMP)

# --- 3. [新语法] 使用 '+' 操作符构建 APE 的基本单元 I' ---
ape_unit = x_pos_env + x_neg_env

print(f"APE 基本单元 (I') 时长: {ape_unit.duration} ns")
print(f"ape_unit 的类型是: {type(ape_unit)}") # 您可以看到它仍然是 CompositeEnvelope

# --- 4. 使用 .repeat() 方法重复 APE 基本单元 ---
n_repeats = 3
ape_sequence_env = ape_unit.repeat(n_repeats)

print(f"重复 {n_repeats} 次后的APE序列总时长: {ape_sequence_env.duration} ns")

# --- 5. 可视化最终的包络波形 ---
t_list = np.linspace(0, ape_sequence_env.duration, 2001)
y_values = ape_sequence_env.value(t_list)

plt.figure(figsize=(12, 6))
plt.plot(t_list, y_values, label=f'APE Sequence Envelope (n={n_repeats})')
for i in range(n_repeats * 2 + 1):
    plt.axvline(x=i * PULSE_DURATION, color='gray', linestyle='--', alpha=0.5)

plt.title("APE Pulse Envelope Constructed with '+' Operator")
plt.xlabel("Time (ns)")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.show()

### 无 DRAG

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, Qobj, destroy
from qutip import Result

# 假设所有必要的类都已从您的库中导入
# from quantum_sim import (DuffingOscillatorModel, Simulator, 
#                          CosineEnvelope, MicrowaveSequence, CompositeEnvelope)

# --- 1. 物理系统和脉冲参数 ---
best_scale = 1.0
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212
PULSE_DURATION = 22 # ns, 单个 pi/2 脉冲的时长
PULSE_AMP = (np.pi / PULSE_DURATION) * best_scale
IDLE_DURATION = PULSE_DURATION # 假设间隔时长与脉冲时长相同

# --- 2. 初始状态与模型/仿真器设置 ---
PSI0 = basis(D_LEVELS, 0)
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

# --- 3. 定义观测量 (X 和 Y 泡利算符) ---
sigmax = basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() + basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag()
sigmay = -1j * (basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag())

# --- 4. 定义扫描和存储参数 ---
n_values = [0, 1, 3, 5] # 重复次数，与论文图1(b)一致
results_xy_lab = {} # 用于存储每个n对应的末态 <X>_lab 和 <Y>_lab

print("所有仿真组件已成功创建。准备开始无DRAG的APE序列仿真...")

# --- 5. 主循环：遍历不同的APE重复次数 n ---
for n in n_values:
    print(f"\n===== 正在计算 n = {n} (无 DRAG) 的情况... =====")

    # a) 使用 '+' 和 '.repeat()' 构建完整的序列包络
    x_pi2_env = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP)
    x_neg_pi2_env = CosineEnvelope(duration=PULSE_DURATION, amp=-PULSE_AMP)
    idle_env = CosineEnvelope(duration=IDLE_DURATION, amp=0.0)
    
    repeating_unit = x_pi2_env + idle_env + x_neg_pi2_env
    full_envelope = x_pi2_env + repeating_unit.repeat(n) if n > 0 else x_pi2_env

    # b) 基于包络创建未经DRAG修正的微波序列
    full_mw_sequence = MicrowaveSequence(envelope=full_envelope, carrier_freq=model.omega_q)
    
    # c) 定义仿真参数并运行单次仿真
    total_duration = full_envelope.duration
    t_list = np.linspace(0, total_duration, int(total_duration * 100 + 1))
    
    result = sim.run(
        psi0=PSI0,
        t_list=t_list,
        sequences={'XY_drive': full_mw_sequence},
        use_rwa=False, # 必须在Lab Frame下
        e_ops=[sigmax, sigmay] # 计算末态的X和Y期望值
    )
    
    # d) 提取并存储末态的 <X>_lab 和 <Y>_lab
    final_x_lab = result.expect[0][-1]
    final_y_lab = result.expect[1][-1]
    results_xy_lab[n] = (final_x_lab, final_y_lab)
    print(f"  完成。末态期望值 (Lab Frame): <X>={final_x_lab:.3f}, <Y>={final_y_lab:.3f}")

print("\n...所有计算完成。开始重构干涉条纹...")

# --- 6. 结果重构与统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')
phi_values = np.linspace(-np.pi, np.pi, 201)

for i, n in enumerate(n_values):
    # a) 从存储的结果中获取 lab frame 的期望值
    final_x_lab, final_y_lab = results_xy_lab[n]
    color = colors(i / len(n_values))
    
    # b) [核心修正] 进行从 Lab Frame 到 Rotating Frame 的坐标逆变换
    total_duration = PULSE_DURATION + n * (PULSE_DURATION + IDLE_DURATION + PULSE_DURATION)
    omega_t = model.omega_q * total_duration
    
    final_x_rot = final_x_lab * np.cos(omega_t) - final_y_lab * np.sin(omega_t)
    final_y_rot = final_x_lab * np.sin(omega_t) + final_y_lab * np.cos(omega_t)
    
    # c) 使用 rotating frame 的期望值重构 P1
    reconstructed_p1 = 0.5 + 0.5 * (-final_y_rot * np.cos(phi_values) + final_x_rot * np.sin(phi_values))
    
    # d) 绘制重构的 P1 vs phi 曲线
    ax.plot(phi_values, reconstructed_p1, '-', color=color, label=f'n = {n}')

# 设置图表格式
ax.set_xlabel("Final Pulse Phase, $\phi$ rad")
ax.set_ylabel("Reconstructed Population of |1> State, $P_1$")
ax.set_title("APE Simulation (No DRAG, with Frame Correction)")
ax.grid(True)
ax.set_ylim(-0.05, 1.05)
ax.legend()
plt.show()

# (可选) 打印出每个n对应的旋转坐标系下的末态相位
print("\n--- Rotating Frame Analysis ---")
for n, (x_lab, y_lab) in results_xy_lab.items():
    total_duration = PULSE_DURATION + n * (PULSE_DURATION + IDLE_DURATION + PULSE_DURATION)
    omega_t = model.omega_q * total_duration
    x_rot = x_lab * np.cos(omega_t) - y_lab * np.sin(omega_t)
    y_rot = x_lab * np.sin(omega_t) + y_lab * np.cos(omega_t)
    
    # 相位是状态在旋转坐标系X-Y平面上的角度
    phase_rad = np.arctan2(y_rot, x_rot)
    print(f"n = {n}: Final state phase in rotating frame = {phase_rad:.4f} radians")

### 有 Half-DRAG

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, Qobj, destroy
from qutip import Result

# 假设所有必要的类都已从您的库中导入
# from quantum_sim import (DuffingOscillatorModel, Simulator, 
#                          CosineEnvelope, MicrowaveSequence, DRAGCorrector)

# --- 1. 物理系统和脉冲参数 ---
best_scale = 1.01
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212
PULSE_DURATION = 20 # ns, 单个 pi/2 脉冲的时长
PULSE_AMP = (np.pi / PULSE_DURATION) * best_scale
IDLE_DURATION = PULSE_DURATION # 假设间隔时长与脉冲时长相同

# --- 2. 初始状态与模型/仿真器设置 ---
PSI0 = basis(D_LEVELS, 0)
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

# --- 3. 定义观测量 (X 和 Y 泡利算符) ---
sigmax = basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() + basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag()
sigmay = -1j * (basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag())

# --- 4. 定义扫描和存储参数 ---
n_values = [0, 1, 3, 5] # 重复次数，与论文图1(c)一致
results_xy_hd = {} # 用于存储每个n对应的末态 <X>_lab 和 <Y>_lab

# --- 5. 创建代表HD协议的DRAG修正器 ---
drag_corrector_hd = DRAGCorrector(anh=model.anh, alpha=0.5)

print("所有仿真组件已成功创建。准备开始带HD修正的APE序列仿真...")

# --- 6. 主循环：遍历不同的APE重复次数 n ---
for n in n_values:
    print(f"\n===== 正在计算 n = {n} (带HD修正) 的情况... =====")

    # a) 使用 '+' 和 '.repeat()' 构建完整的序列包络
    # 序列结构: X(π/2) + n * ( X(π/2) + idle + X(-π/2) )
    x_pi2_env = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP)
    x_neg_pi2_env = CosineEnvelope(duration=PULSE_DURATION, amp=-PULSE_AMP)
    idle_env = CosineEnvelope(duration=IDLE_DURATION, amp=0.0)
    
    repeating_unit = x_pi2_env + idle_env + x_neg_pi2_env
    
    full_envelope = x_pi2_env + repeating_unit.repeat(n) if n > 0 else x_pi2_env

    # b) 基于完整包络创建基础微波序列
    full_mw_sequence = MicrowaveSequence(envelope=full_envelope, carrier_freq=model.omega_q)
    
    # c) 对整个序列应用HD-DRAG修正
    mw_seq_with_drag = drag_corrector_hd.apply(full_mw_sequence)
    
    # d) 定义仿真参数并运行单次仿真
    total_duration = full_envelope.duration
    t_list = np.linspace(0, total_duration, int(total_duration * 100 + 1))
    
    result = sim.run(
        psi0=PSI0,
        t_list=t_list,
        sequences={'XY_drive': mw_seq_with_drag},
        use_rwa=False, # 必须在Lab Frame下
        e_ops=[sigmax, sigmay]
    )
    
    # e) 提取并存储末态的 <X>_lab 和 <Y>_lab
    final_x_lab = result.expect[0][-1]
    final_y_lab = result.expect[1][-1]
    results_xy_hd[n] = (final_x_lab, final_y_lab)
    print(f"  完成。末态期望值 (Lab Frame): <X>={final_x_lab:.3f}, <Y>={final_y_lab:.3f}")

print("\n...所有计算完成。开始重构干涉条纹...")

# --- 7. 结果重构与统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')
phi_values = np.linspace(-np.pi, np.pi, 201)

for i, n in enumerate(n_values):
    # a) 从存储的结果中获取 lab frame 的期望值
    final_x_lab, final_y_lab = results_xy_hd[n]
    color = colors(i / len(n_values))
    
    # b) 进行从 Lab Frame 到 Rotating Frame 的坐标逆变换
    total_duration = PULSE_DURATION + n * (PULSE_DURATION + IDLE_DURATION + PULSE_DURATION)
    omega_t = model.omega_q * total_duration
    
    final_x_rot = final_x_lab * np.cos(omega_t) - final_y_lab * np.sin(omega_t)
    final_y_rot = final_x_lab * np.sin(omega_t) + final_y_lab * np.cos(omega_t)
    
    # c) 使用 rotating frame 的期望值重构 P1
    reconstructed_p1 = 0.5 + 0.5 * (-final_y_rot * np.cos(phi_values) + final_x_rot * np.sin(phi_values))
    
    # d) 绘制重构的 P1 vs phi 曲线
    ax.plot(phi_values, reconstructed_p1, '-', color=color, label=f'n = {n}')

# 设置图表格式
ax.set_xlabel("Final Pulse Phase, $\phi$ rad")
ax.set_ylabel("Reconstructed Population of |1> State, $P_1$")
ax.set_title("APE Simulation with Half-Derivative (HD) Correction")
ax.grid(True)
ax.set_ylim(-0.05, 1.05)
ax.legend()
plt.show()

# (可选) 打印出每个n对应的旋转坐标系下的末态相位
print("\n--- Rotating Frame Analysis ---")
for n, (x_lab, y_lab) in results_xy_hd.items():
    total_duration = PULSE_DURATION + n * (PULSE_DURATION + IDLE_DURATION + PULSE_DURATION)
    omega_t = model.omega_q * total_duration
    x_rot = x_lab * np.cos(omega_t) - y_lab * np.sin(omega_t)
    y_rot = x_lab * np.sin(omega_t) + y_lab * np.cos(omega_t)
    
    # 相位是状态在旋转坐标系X-Y平面上的角度
    phase_rad = np.arctan2(y_rot, x_rot)
    print(f"n = {n}: Final state phase in rotating frame = {phase_rad:.4f} radians")

### 扫描 alpha 系数

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, Qobj, destroy
from qutip import Result

# 假设所有必要的类都已从您的库中导入
# from quantum_sim import (DuffingOscillatorModel, Simulator, 
#                          CosineEnvelope, MicrowaveSequence, DRAGCorrector, CompositeEnvelope)

# --- 1. 物理系统和脉冲参数 ---
best_scale = 1.01
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212
PULSE_DURATION = 20 # ns, 单个 pi/2 脉冲的时长
PULSE_AMP = (np.pi / PULSE_DURATION) * best_scale
IDLE_DURATION = PULSE_DURATION

# --- 2. 初始状态与模型/仿真器设置 ---
PSI0 = basis(D_LEVELS, 0)
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

# --- 3. 定义观测量 (X 和 Y 泡利算符) ---
# [修改] 我们需要同时测量 X 和 Y
sigmax = basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() + basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag()
sigmay = -1j * (basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag())

# --- 4. 定义扫描和存储参数 ---
n_values = [0, 1, 3]
scanned_alphas = np.linspace(-1.0, 2.0, 31)
results_dict = {}

print("所有仿真组件已成功创建。准备开始APE序列的DRAG Alpha扫描...")

# --- 5. 主循环：遍历不同的APE重复次数 n ---
for n in n_values:
    print(f"\n===== 开始扫描 n = {n} 的情况... =====")

    # a) 定义本次扫描的 Setup 函数
    def ape_drag_setup(alpha: float) -> dict:
        x_pi2_env = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP)
        x_neg_pi2_env = CosineEnvelope(duration=PULSE_DURATION, amp=-PULSE_AMP)
        idle_env = CosineEnvelope(duration=IDLE_DURATION, amp=0.0)
        repeating_unit = x_pi2_env + idle_env + x_neg_pi2_env
        full_envelope = x_pi2_env + repeating_unit.repeat(n) if n > 0 else x_pi2_env
        full_mw_sequence = MicrowaveSequence(envelope=full_envelope, carrier_freq=model.omega_q)
        drag_corrector = DRAGCorrector(anh=model.anh, alpha=alpha)
        dragged_sequence = drag_corrector.apply(full_mw_sequence)
        return {'sequences': {'XY_drive': dragged_sequence}}

    # b) [修改] Post-process 函数，现在返回一个包含 (<X>_lab, <Y>_lab) 的元组
    def final_xy_lab_processor(result: Result) -> tuple:
        final_x = result.expect[0][-1]
        final_y = result.expect[1][-1]
        return (final_x, final_y)

    # c) 设置本次扫描的固定参数和 Scanner
    total_duration = PULSE_DURATION + n * (PULSE_DURATION + IDLE_DURATION + PULSE_DURATION)
    t_list = np.linspace(0, total_duration, int(total_duration * 100 + 1))
    # [修改] e_ops 现在包含 sigmax 和 sigmay
    base_args = {'psi0': PSI0, 't_list': t_list, 'use_rwa': False, 'e_ops': [sigmax, sigmay]}
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 执行对 alpha 的扫描
    _, final_xy_lab_values = scanner.run(
        sweep_values=scanned_alphas,
        setup_function=ape_drag_setup,
        post_process_function=final_xy_lab_processor,
        description=f"Scanning Alpha for n={n}"
    )
    results_dict[n] = final_xy_lab_values

print("\n...所有扫描完成。")

# --- 6. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

for i, n in enumerate(n_values):
    # a) 获取 lab frame 结果并解包
    final_x_lab, final_y_lab = np.array(results_dict[n]).T
    
    # b) [核心修正] 进行从 Lab Frame 到 Rotating Frame 的坐标逆变换
    total_duration = PULSE_DURATION + n * (PULSE_DURATION + IDLE_DURATION + PULSE_DURATION)
    omega_t = model.omega_q * total_duration
    
    final_x_rot = final_x_lab * np.cos(omega_t) - final_y_lab * np.sin(omega_t)
    final_y_rot = final_x_lab * np.sin(omega_t) + final_y_lab * np.cos(omega_t)
    
    # c) 我们关心的保真度指标是 -<Y>_rot
    fidelity_proxy = -final_y_rot

    color = colors(i / len(n_values))
    ax.plot(scanned_alphas, fidelity_proxy, 'o-', ms=5, color=color, label=f'n = {n} repetitions')
    
    optimal_alpha_index = np.argmax(fidelity_proxy)
    optimal_alpha = scanned_alphas[optimal_alpha_index]
    max_fidelity = fidelity_proxy[optimal_alpha_index]
    ax.plot(optimal_alpha, max_fidelity, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Fidelity Proxy, $-<Y>_{rot}$")
ax.set_title("DRAG Alpha Calibration using APE Sequence (Corrected)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
n_values = list(range(0, 20, 2))

In [ ]:
n_values 

In [ ]:
20 / (1/ 5.3)

In [ ]:
107 / 5.3

## 5. 研究改变 Pi/2 脉冲的时间的影响 （lab 坐标系）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, Qobj, destroy
from qutip import Result

# 假设所有必要的类都已从您的库中导入
# from quantum_sim import ...

# --- 1. 物理系统参数 ---
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# --- 2. 初始状态与模型/仿真器设置 ---
PSI0 = basis(D_LEVELS, 0)
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

# --- 3. 定义观测量 (Y 泡利算符) ---
sigmay = -1j * (basis(D_LEVELS, 0) * basis(D_LEVELS, 1).dag() - basis(D_LEVELS, 1) * basis(D_LEVELS, 0).dag())

# --- 4. 定义扫描参数 ---
# 扫描脉冲时长，例如从20ns到21ns，这个范围内应包含多个振荡周期
duration_scan_values = np.linspace(20.0, 21.0, 201)

# --- 5. 配置并运行 Scanner ---
# 定义固定的仿真参数
base_args = {
    'psi0': PSI0,
    'use_rwa': False,
    'e_ops': [sigmay]
}
scanner = Scanner(simulator=sim, base_sim_args=base_args)

# a) Setup 函数: 根据不同的时长，动态创建脉冲和时间轴
def duration_setup(duration: float) -> dict:
    # 保持脉冲面积为 pi/2，反算出对应的幅度
    amp = np.pi / duration
    envelope = CosineEnvelope(duration=duration, amp=amp)
    sequence = MicrowaveSequence(envelope=envelope, carrier_freq=model.omega_q)
    
    # 仿真时间轴也必须是动态的
    t_list = np.linspace(0, duration, int(duration * 100 + 1))
    
    # 返回本次扫描需要更新的参数
    return {
        'sequences': {'XY_drive': sequence},
        't_list': t_list
    }

# b) Post-process 函数: 提取末态的 <Y>
def final_y_processor(result: Result) -> float:
    return result.expect[0][-1]

print("开始扫描脉冲时长...")
scanned_durations, final_y_values = scanner.run(
    sweep_values=duration_scan_values,
    setup_function=duration_setup,
    post_process_function=final_y_processor,
)
print("...扫描完成。")

# --- 6. 结果可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))

# 绘制仿真结果
ax.plot(scanned_durations, final_y_values, 'o', ms=4, label='Simulated Final $<Y>$')

# # 绘制理论预测曲线 y(t) = -cos(omega_q * t)
y_theory = -np.cos(model.omega_q * np.array(scanned_durations))
ax.plot(scanned_durations, y_theory, '-', lw=2, color='r', label='Theoretical Prediction: $-\cos(\omega_q t)$')

# 设置图表格式
ax.set_xlabel("Pulse Duration, t (ns)")
ax.set_ylabel("Final Expectation Value, $<Y>$")
ax.set_title("Final $<Y>$ vs. Pulse Duration in Lab Frame")
ax.grid(True)
ax.legend()
plt.show()

所以直接的结论是，使用lab frame进行仿真时，要注意从lab frame切换回rotating frame